#  Table 2  1993 QS fe_13 data

#  perform a brute-force chi-squared minimization and analyze the 

# search data

In [ ]:
import os
import json
import pickle
import fnmatch
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import ChiantiPy.core as ch
import ChiantiPy.tools.util as chutil
import ChiantiPy.model as mdl
import ChiantiPy.model.Maker as mm
import ChiantiPy.Gui as chGui

In [ ]:
%autoreload 3

In [ ]:
matplotlib qt

In [ ]:
pwd

In [ ]:
ourDir = os.getcwd()

In [ ]:
ourDir

In [ ]:
myIon = 'fe_13'

In [ ]:
nameDict = chutil.convertName(myIon)

In [ ]:
nameDict.keys()

In [ ]:
nameDict['spectroscopic']

In [ ]:
nameDict['experimental']

In [ ]:
ls

In [ ]:
dirList = os.listdir(ourDir)

In [ ]:
len(dirList)

In [ ]:
jsonList = []
for fname in dirList:
    if fnmatch.fnmatch(fname, '*.json'):
        jsonList.append(fname)

In [ ]:
for idx, fname in enumerate(jsonList):
    print('idx:  %i  fname:  %s'%(idx, fname))

## fe_13 intensities are in the .json file

In [ ]:
jsonName = jsonList[0]

In [ ]:
with open(jsonName, 'r') as inpt:
    specData = json.load(inpt)

In [ ]:
specData['ref']

In [ ]:
specData.keys()

## in notebook #1, created the 'match' pickle file

In [ ]:
wghtFactor = 0.2

In [ ]:
box = mm.maker(specData, wghtFactor=wghtFactor, verbose = True)

## the match pickle file is tab2_1993_qs_fe_13_match.pkl

In [ ]:
pklList = []
for fname in dirList:
    if fnmatch.fnmatch(fname, '*.pkl'):
        pklList.append(fname)

In [ ]:
for idx, fname in enumerate(pklList):
    print('idx:  %i  fname:  %s'%(idx, fname))

In [ ]:
matchName = pklList[0]

In [ ]:
matchName

In [ ]:
box.loadMatch(matchName)

In [ ]:
plt.figure()
box.emPlot(vs='d')
plt.text(1.e+7, 3.e+26, nameDict['spectroscopic'], horizontalalignment='center', fontsize=16)
plt.xlabel('Electron Density (cm$^{-3}$)', fontsize=14)
plt.ylabel('Emissin Measure (cm$^{-5}$)', fontsize=14)
plt.tight_layout()

In [ ]:
print(' # of density values %5i'%(box.EDensity.size))

## from the plot, a density of 7.e+8  and an em of 1.e+27 is estimated

In [ ]:
Dindex = np.argmin(np.abs(box.EDensity - 7.e+8))
' Dindex = %5i'%(Dindex)

In [ ]:
box.emSetIndices(Dindex)
print('density set to %10.2e'%(box.EDensity[Dindex]))

In [ ]:
em = 1.e+27
emLog = np.log10(em)
' em  %10.2e  emLog %10.2f'%(em, emLog)

In [ ]:
box.emSet(emLog)

## can reset the WghtFactor

In [ ]:
box.WghtFactor

In [ ]:
box.WghtFactor = 0.2

In [ ]:
box.predict()

In [ ]:
box.predictPrint(filename='tab2_fe_13_priorPredictPrint.txt')

## brute force search of 1D density space

## if the searchData has already been pickled, it can be reloaded and the search skipped

In [ ]:
box.search1dSpace(emLog, verbose=1)

## pickle the box with search data

In [ ]:
box.SearchData.keys()

In [ ]:
box.SearchData['best'].keys()

In [ ]:
searchDataFile = os.path.splitext(matchName)[0] + '_1dsearch.pkl'

In [ ]:
searchDataFile

In [ ]:
box.saveSearchData(searchDataFile)

In [ ]:
ls

## have created a pickle of the searchData dict

## open the pickled search data

In [ ]:
mydir = os.getcwd()
mylist = os.listdir('.')
newlist = fnmatch.filter(mylist,'*.pkl')
selected = chGui.gui.selectorDialog(newlist,'pick one')

In [ ]:
mypkl = selected.selectedText[0]

In [ ]:
print(mypkl)

In [ ]:
with open(mypkl, 'rb') as inpt:
    searchData = pickle.load(inpt)

## to reload the searchData into the box class - needed for predictPrint

## can recontinue or restart analysis

In [ ]:
box.loadSearchData(mypkl)

## or just continue here

In [ ]:
Didx = searchData['best']['idx']
emLog = searchData['best']['emfit']
em = searchData['best']['em']
print(' minimum chisq achieved for D index %5i %10.2e and EM %10.3f %10.3e'%(Didx, box.EDensity[Didx],emLog, em))

In [ ]:
searchData.keys()

In [ ]:
searchData['best'].keys()

In [ ]:
'Chisq  %10.2f  reduced Chisq = %10.2f '%(searchData['best']['chisq'], searchData['best']['reducedChisq'])

In [ ]:
minChisq = searchData['best']['chisq']
maxChisq = max(searchData['chisq'])
' min, max of chisq = %10.2f %10.2f'%(minChisq, maxChisq)

In [ ]:
box.Nparams

In [ ]:
nObs = len(box.Match)
dof = nObs - box.Nparams

In [ ]:
' nObs:  %i  nParmas:  %6.1f  degrees of freedom (dof):  %6.1f'%(nObs, box.Nparams, dof)

In [ ]:
q = [.32, 0.05]
chi2 = stats.chi2.isf(q = q , df = dof)
' Chi2 for confidence levels %10.2f  %10.2f   = %10.2f %10.2f '%(1.-q[0],1.-q[1],chi2[0],chi2[1])

In [ ]:
plt.figure()
plt.semilogx(searchData['densSearched'],searchData['chisq'], 'k', lw=2)
xy = plt.axis()
plt.ylim(0.,xy[3])
plt.axhline(minChisq + chi2[0], color = 'r', label='65%')
plt.axhline(minChisq + chi2[1], color = 'b', label = '95%')
plt.legend(loc='upper center', fontsize=14)
plt.xlabel('Electron Density (cm$^{-3}$)',fontsize=14)
plt.ylabel('Chi-squared',fontsize=14)
plt.tight_layout()

In [ ]:
chisq = np.asarray(searchData['chisq'])

In [ ]:
good0 = chisq < minChisq + chi2[0]

In [ ]:
good1 = chisq < minChisq + chi2[1]

In [ ]:
dens68 = box.EDensity[good0].min(),box.EDensity[good0].max()
' mean Density range of  68 per cent confidence:  %10.2e to %10.2e'%(dens68)

In [ ]:
dens95 = box.EDensity[good1].min(),box.EDensity[good1].max()
' mean Density range of  95 per cent confidence:  %10.2e to %10.2e'%(dens95)

In [ ]:
densBest = box.EDensity[searchData['best']['idx']]
' best fit Density %10.2e'%(densBest)

In [ ]:
em68 = [searchData['em'][good0].min(),searchData['em'][good0].max()]
' mean EM range of  68 per cent confidence:  %10.2e to %10.2e '%(em68[0], em68[1])

In [ ]:
em95 = [searchData['em'][good1].min(),searchData['em'][good1].max()]
' mean Density range of  95 per cent confidence:  %10.2e to %10.2e'%(em95[0], em95[1])

In [ ]:
plt.figure()
box.emPlot(vs='d')
xy = plt.axis()
plt.axvline(densBest,color='k',lw=2,label='best fit')
plt.axvline(dens68[0], color = 'r', linestyle = '--', lw=2, label='68% confidence')
plt.axvline(dens68[1], color = 'r', linestyle = '--', lw=2)
plt.axvline(dens95[0], color = 'b', linestyle = '--', lw=2, label='95% confidence')
plt.axvline(dens95[1], color = 'b', linestyle = '--', lw=2)
plt.xlabel('Electron Density (cm$^{-3}$)',fontsize=14)
plt.ylabel('Chi-squared',fontsize=14)
plt.text(2.e+10,4.e+26, nameDict['spectroscopic'] ,fontsize=16)
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()

In [ ]:
searchData['best']['idx']

In [ ]:
box.emSetIndices(searchData['best']['idx'])
print('dens set to %10.2e'%(searchData['best']['idx']))

In [ ]:
searchData['best']['emfit']

In [ ]:
box.emSet(searchData['best']['emfit'])

In [ ]:
box.predict()

In [ ]:
box.predictPrint(minContribution=0.01, filename='tab2_fe_13_postPredictPrint.txt')

In [ ]:
hasattr(box, 'SearchData')

In [ ]:
box.diffPrint(filename='tab2_fe_13_postDiffPrint.txt')

In [ ]:
box.Diff.keys()

## write the summary

In [ ]:
summaryFile = os.path.splitext(mypkl)[0] + '_summary.txt'

In [ ]:
summaryFile

In [ ]:
diff = box.Diff
with open(summaryFile,'w') as outpt:
    outpt.write(' chisq minimum = %10.2e \n'%(box.SearchData['best']['chisq']))
    outpt.write(' reduced chisq minimum = %10.2e \n'%(box.SearchData['best']['reducedChisq']))
    outpt.write(' index         = %5i \n'%(box.SearchData['best']['idx']))
    outpt.write(' log EM        = %10.3e \n'%(box.SearchData['best']['emfit']))
    outpt.write(' EM            = %10.3e \n'%(10.**box.SearchData['best']['emfit']))
    outpt.write(' Density       = %10.3e  \n'%(box.SearchData['best']['density']))
    outpt.write(' mean of diff = %10.3f std = %10.3f \n'%(np.mean(diff['diff']), np.std(diff['diff'])))

### a dict of the summary data

In [ ]:
tab2_fe_13_br = {}
tab2_fe_13_br['d'] = {'temp': box.SearchData['best']['temperature']}
tab2_fe_13_br['em'] = {'temp': box.SearchData['best']['temperature']}
tab2_fe_13_br['d']['ionS'] = 'fe_13'
tab2_fe_13_br['em']['ionS'] = 'fe_13'
tab2_fe_13_br['d']['best'] = box.SearchData['best']['density']
tab2_fe_13_br['d']['mean'] = box.SearchData['best']['density']
tab2_fe_13_br['d']['idx'] = box.SearchData['best']['idx']
tab2_fe_13_br['d']['68'] = dens68
tab2_fe_13_br['d']['95'] = dens95
tab2_fe_13_br['em']['best'] = box.SearchData['best']['em']
tab2_fe_13_br['em']['mean'] = box.SearchData['best']['em']
tab2_fe_13_br['em']['idx'] = box.SearchData['best']['idx']
tab2_fe_13_br['em']['68'] = em68
tab2_fe_13_br['em']['95'] = em95

### pickle summary dict

In [ ]:
with open('tab2_fe_13_br_summary.pkl','wb') as outpt:
    pickle.dump(tab2_fe_13_br, outpt)

In [ ]:
with open('tab2_fe_13_br_summary.pkl','rb') as inpt:
    tab2_fe_13_br_summary = pickle.load(inpt)

In [ ]:
tab2_fe_13_br_summary.keys()

In [ ]:
tab2_fe_13_br_summary['d'].keys()

In [ ]:
tab2_fe_13_br_summary['em'].keys()

## plot the ave, std, errors

In [ ]:
wvl = box.Diff['wvl']
diff = box.Diff['diffOverInt']

In [ ]:
diffMean = diff.mean()
diffStd = diff.std()

In [ ]:
' diffAve %10.3f  diff Std  %10.3f'%(diffMean, diffStd)

In [ ]:
plt.figure()
plt.plot(wvl, diff,'o')

In [ ]:
plt.axhline(diffMean, color='k', lw=2, label='Mean')
plt.axhline(diffMean + diffStd, color='r', lw=2, linestyle='--', label='1 std')
plt.axhline(diffMean - diffStd, color='r', lw=2, linestyle='--')  #, label='1 std')
plt.axhline(diffMean + 2.*diffStd, color='b', lw=2, linestyle='dotted', label='2 std')
plt.axhline(diffMean - 2.*diffStd, color='b', lw=2, linestyle='dotted')  #, label='2 std')
plt.axhline(diffMean + 3.*diffStd, color='g', lw=2, linestyle='dotted', label='3 std')
plt.axhline(diffMean - 3.*diffStd, color='g', lw=2, linestyle='dotted')  #, label='3 std')

In [ ]:
plt.xlabel('Wavelength ($\AA$)', fontsize=14)
plt.ylabel(r'(Obs - Pred)/(w $\times$ Obs)', fontsize=14)

In [ ]:
plt.title(' mean:  %8.3f  std:  %8.3f wght: %8.3f'%(diffMean, diffStd, wghtFactor), fontsize=14)

In [ ]:
plt.legend(loc='upper right', bbox_to_anchor=(0.99, 1.0), fontsize=12)

In [ ]:
plt.tight_layout()